In [ ]:
group_227 = group_sg.get_group(227)

def plot_feature_import(data, n_features):
    exclude_feature = ['Eg', 'Ef', 'alpha_r', 'beta_r', 'gamma_r', 'x_In', 'x_Al',]
    features = list(df.drop(exclude_feature, axis=1))
    X = data.drop(exclude_feature, axis=1).values
    y = data['Eg']

    selector = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=0).fit(X, y)

    importances = selector.feature_importances_
    indices = np.argsort(importances)[::-1]
    sorted_features = [features[idx] for idx in indices]
#    select_features = sorted_features[:n_features]
    
    fig1 = plt.figure(figsize=(8, 6))
    plt.subplot()
    plt.bar(range(X.shape[1]), importances[indices], align="center")
    plt.xticks(range(X.shape[1]), sorted_features, rotation=90)
    
    fig2 = plt.figure(figsize=(8,6*n_features))
    for i in range(n_features):
        a = str(n_features)+'1'+str(i)
        plt.subplot(a)
        plt.xlabel(sorted_features[i])
        plt.scatter(data[sorted_features[i]], data['Eg'], alpha=0.5)
        
    plt.show()
    
    
plot_feature_import(gamma2, 4)

from sklearn.model_selection import train_test_split

n_features = None
select_features = sorted_features[:n_features]

X = group_227[select_features].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

rfr = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=99)
rfr.fit(X_train, y_train)

y_train_pred = rfr.predict(X_train)
y_test_pred = rfr.predict(X_test)

print(rmsle(y_train, y_train_pred), rmsle(y_test, y_test_pred))

corr = df_train.corr()
fig, ax = plt.subplots(figsize=(10,8))

sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values, ax=ax)

## Let's try some more complex models

from sklearn.model_selection import train_test_split

n_features = None
select_features = sorted_features[:n_features]

X = df[select_features].values
X_train, X_test, y_train, y_test = train_test_split(X, y_g, test_size=0.2, random_state=99)


rfr = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=99)
rfr.fit(X_train, y_train)

y_train_pred = rfr.predict(X_train)
y_test_pred = rfr.predict(X_test)

print(rmsle(y_train, y_train_pred), rmsle(y_test, y_test_pred))

from sklearn.model_selection import GridSearchCV


n_est = np.arange(10, 210, 40)
n_depth = np.arange(3, 21, 5)

param_grid = dict(n_estimators=n_est, max_depth=n_depth)
randomf = GridSearchCV(RandomForestRegressor(random_state=99), 
                       param_grid=param_grid, 
                       cv=3)

randomf.fit(X, y)

#display(pd.DataFrame(randomf.cv_results_))
print("Best estimator:",randomf.best_estimator_)

Best_rf = randomf.best_estimator_
print("Best parameter", randomf.best_params_)
#print("Best performance:", Best_rf.score(X_test, y_test))


scores = randomf.cv_results_['mean_test_score']
scores = np.array(scores).reshape(len(n_est), len(n_depth))

plt.figure(figsize=(8, 6))
plt.imshow(scores, interpolation='nearest',cmap=plt.cm.coolwarm)
plt.xlabel('n_est')
plt.ylabel('n_depth')
plt.colorbar()
plt.xticks(np.arange(len(n_est)), n_est, rotation=45)
plt.yticks(np.arange(len(n_depth)), n_depth)
plt.show()

## Cross_validate

from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import make_scorer
scoring = make_scorer(rmsle)

#score = cross_val_score(rfr, X, y, cv=5)
cv = cross_validate(rfr, X, y, cv=5, scoring=scoring, return_train_score=False)
cv



features = list(df.drop(exclude_feature, axis=1))

X = df.drop(exclude_feature, axis=1).values

selector = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=0).fit(X, y_g)

importances = selector.feature_importances_
indices = np.argsort(importances)[::-1]
sorted_features = [features[idx] for idx in indices]

plt.figure(figsize=(16,6))
plt.subplot(121)
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), sorted_features, rotation=45)

plt.subplot(122)
plt.xlabel('atomic density')
plt.ylabel('bandgap energy (ev)')
plt.scatter(df['atomic_density'], df['Eg'], alpha=0.5)
plt.show()

## Simply polynomial regression

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

select_features = ['atomic_density']
X = df[select_features].values
y = y_g

model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])

model = model.fit(X, y)
y_pred = model.predict(X)

print("Coef:", model.named_steps['linear'].coef_)
print("RMSLE:", rmsle(y, y_pred))

#X_plot = np.linspace(min(X.T[0]),max(X.T[0]),100)
#y_plot = model.predict(X_plot[:,np.newaxis])

plt.figure(figsize=(16,6))
plt.subplot(121)
plt.xlabel('actual bandgap')
plt.ylabel('predicted bandgap')
plt.scatter(y, y_pred, alpha=0.5)
plt.plot([0, 5], [0, 5], lw=2, color='teal')

plt.subplot(122)
plt.xlabel('y-y_pred')
plt.hist(y-y_pred, bins=25, alpha=0.7)

plt.show()

select_features = ['atomic_density', 'c', 'diff_In_Al', 'vol']
X = df[select_features].values
model = model.fit(X, y)
y_pred = model.predict(X)

print("Coef:", model.named_steps['linear'].coef_)
print("RMSLE:", rmsle(y, y_pred))

plt.figure(figsize=(16,6))
plt.subplot(121)
plt.xlabel('actual bandgap')
plt.ylabel('predicted bandgap')
plt.scatter(y, y_pred, alpha=0.5)
plt.plot([0, 5], [0, 5], lw=2, color='teal')

plt.subplot(122)
plt.xlabel('y-y_pred')
plt.hist(y-y_pred, bins=25, alpha=0.7)

plt.show()

# use random forests to quantify the importances of each feature

# list of columns not to be used for training
non_features = ['id', 'Ef', 'Eg',
               'alpha_r', 'beta_r', 'gamma_r']

# list of columns to be used for training each model
features = [col for col in list(df) if col not in non_features]
print('%i features: %s' % (len(features), features))

# make feature matrix
X = df[features].values

# make target columns for each target property
y_E = df['Ef'].values
y_Eg = df['Eg'].values

# split into training and test for the purposes of this demonstration
test_size = 0.2
rstate = 42
X_train_Ef, X_test_Ef, y_train_Ef, y_test_Ef = train_test_split(X, y_Ef, 
                                                            test_size=test_size,
                                                            random_state=rstate)
X_train_Eg, X_test_Eg, y_train_Eg, y_test_Eg = train_test_split(X, y_Eg, 
                                                                test_size=test_size, 
                                                                random_state=rstate)

# number of base decision tree estimators
n_est = 100
# maximum depth of any given decision tree estimator
max_depth = 5
# random state variable
rstate = 42
# initialize a random forest algorithm
rf_Ef = RandomForestRegressor(n_estimators=n_est, 
                             max_depth=max_depth,
                             random_state=rstate)
rf_Eg = RandomForestRegressor(n_estimators=n_est, 
                             max_depth=max_depth,
                             random_state=rstate)
# fit to training data
rf_Ef.fit(X_train_Ef, y_train_Ef)
rf_Eg.fit(X_train_Eg, y_train_Eg)

# evaluate performance of the random forest models

def rmsle(actual, predicted):
    """
    Args:
        actual (1d-array) - array of actual values (float)
        predicted (1d-array) - array of predicted values (float)
    Returns:
        root mean square log error (float)
    """
    return np.sqrt(np.mean(np.power(np.log1p(actual)-np.log1p(predicted), 2)))

def plot_actual_pred(train_actual, train_pred, 
                     test_actual, test_pred,
                     target):
    """
    Args:
        train_actual (1d-array) - actual training values (float)
        train_pred (1d-array) - predicted training values (float)
        test_actual (1d-array) - actual test values (float)
        test_pred (1d-array) - predicted test values (float)
        target (str) - target property
    Returns:
        matplotlib scatter plot of actual vs predicted
    """
    s = 75
    lw = 0
    alpha = 0.2
    train_color = 'orange'
    train_marker = 's'
    test_color = 'red'
    test_marker = '^'
    axis_width = 1.5
    maj_tick_len = 6
    fontsize = 16
    label = '__nolegend__'
    ax = plt.scatter(train_pred, train_actual,
                     marker=train_marker, color=train_color, s=s, 
                     lw=lw, alpha=alpha, label='train')
    ax = plt.scatter(test_pred, test_actual,
                     marker=test_marker, color=test_color, s=s, 
                     lw=lw, alpha=alpha, label='test')
    ax = plt.legend(frameon=False, fontsize=fontsize, handletextpad=0.4)    
    all_vals = list(train_pred) + list(train_actual) + list(test_pred) + list(test_actual)
    full_range = abs(np.max(all_vals) - np.min(all_vals))
    cushion = 0.1
    xmin = np.min(all_vals) - cushion*full_range
    xmax = np.max(all_vals) + cushion*full_range
    ymin = xmin
    ymax = xmax    
    ax = plt.xlim([xmin, xmax])
    ax = plt.ylim([ymin, ymax])
    ax = plt.plot([xmin, xmax], [ymin, ymax], 
                  lw=axis_width, color='black', ls='--', 
                  label='__nolegend__')
    ax = plt.xlabel('predicted ' + target, fontsize=fontsize)
    ax = plt.ylabel('actual ' + target, fontsize=fontsize)
    ax = plt.xticks(fontsize=fontsize)
    ax = plt.yticks(fontsize=fontsize)
    ax = plt.tick_params('both', length=maj_tick_len, width=axis_width, 
                         which='major', right=True, top=True)
    return ax  

y_train_E_pred = rf_Ef.predict(X_train_Ef)
y_test_E_pred = rf_Ef.predict(X_test_Ef)
target_E = 'formation energy (eV/atom)'
print('RMSLE for formation energies = %.3f eV/atom (training) and %.3f eV/atom (test)' 
      % (rmsle(y_train_Ef, y_train_E_pred),  (rmsle(y_test_Ef, y_test_E_pred))))
y_train_Eg_pred = rf_Eg.predict(X_train_Eg)
y_test_Eg_pred = rf_Eg.predict(X_test_Eg)
target_Eg = 'band gap (eV)'
print('RMSLE for band gaps = %.3f eV (training) and %.3f eV (test)' 
      % (rmsle(y_train_Eg, y_train_Eg_pred), (rmsle(y_test_Eg, y_test_Eg_pred))))
fig4 = plt.figure(4, figsize=(11,5))
ax1 = plt.subplot(121)
ax1 = plot_actual_pred(y_train_E, y_train_E_pred,
                       y_test_E, y_test_E_pred,
                       target_E)
ax2 = plt.subplot(122)
ax2 = plot_actual_pred(y_train_Eg, y_train_Eg_pred,
                       y_test_Eg, y_test_Eg_pred,
                       target_Eg)
plt.tight_layout()
plt.show()
plt.close()